# Import Module / Library

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv

# Initialize Mediapipe 

In [2]:
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

# Create Header Dataset

In [3]:
landmarks = ["class", "sudut_kiri", "sudut_kanan"]

In [4]:
with open('sudut_hip.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Create Function Calculate Angle

In [5]:
def hitung_sudut(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    sudut = np.abs(radians * 180.0 / np.pi)
    
    if sudut > 180.0:
        sudut = 360 - sudut
        
    return sudut

# Create Function Store Angle

In [6]:
def export_sudut(results, label):
    try:
        # Ekstraksi koordinat yang diperlukan
        coords = results.pose_landmarks.landmark
        titik_11 = [coords[11].x, coords[11].y]
        titik_12 = [coords[12].x, coords[12].y]
        titik_23 = [coords[23].x, coords[23].y]
        titik_24 = [coords[24].x, coords[24].y]
        titik_25 = [coords[25].x, coords[25].y]
        titik_26 = [coords[26].x, coords[26].y]

        # Hitung sudut kiri dan kanan
        sudut_kiri = hitung_sudut(titik_11, titik_23, titik_25)
        sudut_kanan = hitung_sudut(titik_12, titik_24, titik_26)

        row = [label, sudut_kiri, sudut_kanan]

        with open('sudut_hip.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
    except Exception as e:
        print(f"Error: {e}")
        pass

# Extract Video to Dataset

In [ ]:
cap = cv2.VideoCapture("bottom_pos.mp4")

#Initiate Holistic Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # streaming the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to the desired window size
        frame = cv2.resize(frame, (640, 480))

        # recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detections
        results = pose.process(image)
    
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        k = cv2.waitKey(1)
        if k == ord('b'):  # Press 'u' for 'up'
            export_sudut(results, 1)
        elif k == ord('s'):  # Press 'd' for 'down'
            export_sudut(results, 0)
        elif k == ord('q'):  # Press 'q' to quit
            break
        # Stream vid result
        cv2.imshow("Raw Cam Feed", image)

    cap.release()
    cv2.destroyAllWindows()